This notebook explores a mock dataset of stars, constellations, galaxies, and users. It demonstrates how I approach data analysis: cleaning, descriptive stats, inferential tests, and visualization — even in a science-inspired domain. Unlike my other projects, this project is focused on **statistics and analytical depth**.

This notebook will focus on **data loading and cleanup**.

First, load the CSVs into dataframes and display a few rows 

In [22]:
import pandas as pd

datasets = [
    ["users", None],
    ["galaxies", None],
    ["constellations", None],
    ["stars", None]
    ]

for i, [table, _] in enumerate(datasets):
    df = pd.read_csv(f"data/{table}.csv")
    print(f"\n=== Table: {table} (shape: {df.shape}) ===")
    display(df.head(3))
    # save the dataframe back into the list, for later use
    datasets[i][1] = df


=== Table: users (shape: (120, 7)) ===


,user_id,username,email,password,first_name,last_name,date_of_birth
0,1,taylorgarcia1,taylorgarcia1@example.com,REDACTED,Taylor,Garcia,1999-04-15
1,2,drewtorres2,drewtorres2@example.com,REDACTED,Drew,Torres,1995-08-21
2,3,caseylopez3,caseylopez3@example.com,REDACTED,Casey,Lopez,1999-10-19



=== Table: galaxies (shape: (121, 9)) ===


,galaxy_id,galaxy_name,galaxy_type,distance_mly,redshift,mass_solar,diameter_ly,added_by,verified_by
0,1,Galaxy-001,Irregular,251.0,25.0,981,15475,85,119.0
1,2,Galaxy-002,Spiral,612.0,61.0,25,91959,95,118.0
2,3,Galaxy-003,Elliptical,55.0,6.0,485,202910,2,113.0



=== Table: constellations (shape: (152, 5)) ===


,constellation_id,constellation_name,galaxy_id,added_by,verified_by
0,1,Constellation-001,91,81,119
1,2,Constellation-002,57,70,104
2,3,Constellation-003,101,82,108



=== Table: stars (shape: (172, 12)) ===


,star_id,star_name,star_type,constellation_id,right_ascension,declination,apparent_magnitude,spectral_type,distance_mly,temperature_k,added_by,verified_by
0,1,Star-0001,Main Sequence,112,84.483,69.068,6.56,G9,223.0,6775,76,97
1,2,Star-0002,Giant,113,60.135,-59.829,4.37,B4,667.0,19427,44,120
2,3,Star-0003,Main Sequence,29,311.118,10.465,7.68,K3,388.0,4164,35,102


In real projects, not all “messy” data should be cleaned automatically.

- Some issues are safe to fix (e.g., casing, whitespace, duplicate rows).
- Others, like potential typos or suspicious values, can’t be corrected without a reference list or domain knowledge.

In this notebook, I’ll demonstrate both:

- Cleaning where it’s safe,
- And flagging suspect values that would normally be escalated to a data owner.

For these datasets, the business rules are as follows:

- There must not be **any** blank fields.
- There must not be any duplicate records.
- Names of users, galaxies, constellations and stars must not have any leading nor trailing spaces.
- Names must be in the proper case i.e. uppercase for first letters of words, lowercase for the rest.
- Values for `added_by` and `verified_by` must exist in **users**.`user_id`.
- Values for **stars**.`constellation_id` must exist in **constellations**.`constellation_id`.
- Values for **constellations**.`galaxy_id` must exist in **galaxies**.`galaxy_id`.
- In **stars**, `apparent_magnitude` must be between 0 and 10; `distance_mly` must be between 0 and 1000.
- In **galaxies**, `redshift` must be between 0 and 180; `distance_mly` must be between 0 and 1000.

In [23]:
# Do the straightforward cleanup steps first
for i, [_, df] in enumerate(datasets):
    if i == 0:
        # no need to modify the **users** dataframe
        continue
    df = df.map(lambda v: v.strip() if isinstance(v, str) else v)  # trim spaces
    df = df.map(lambda v: v.title() if isinstance(v, str) else v)  # fix case issues
    columns_without_pk = list(df)[1:]
    df = df.drop_duplicates(subset=columns_without_pk)  # remove duplicates
    df = df.fillna("")
    # save the dataframe back into the list, for later use
    datasets[i][1] = df

In [24]:
# Do dataframe-specific cleanup

df = datasets[0][1]
# no cleanup needed for **users** so just retrieve the user_ids
set_of_user_ids = set(df["user_id"])
df["remarks"] = "ALL GOOD"
datasets[0][1] = df

# for all other dataframes:
# create an empty list, to note down remarks
# iterate through rows.
# for each row:
#   analyze each field in the row for issues and flag them
#   if there are no issues, mark as good
#   append the list of issues to the list of remarks
# append the remarks to the dataframe as a new column


In [25]:
# for galaxies
df = datasets[1][1]
list_of_remarks = []
set_of_galaxy_ids = set()
df_columns = list(df)

for row in df.itertuples():
    row_remarks = []
    for column in df_columns:
        if getattr(row, column) == "":
            row_remarks.append(f"Value missing for {column}")

    if row.redshift and not 0 <= float(row.redshift) <= 180:
        row_remarks.append("Invalid value for redshift")
    if row.distance_mly and not 0 <= float(row.distance_mly) <= 1000:
        row_remarks.append("Invalid value for distance_mly")
    if row.added_by and int(row.added_by) not in set_of_user_ids:
        row_remarks.append("Invalid value for added_by")
    if row.verified_by and int(row.verified_by) not in set_of_user_ids:
        row_remarks.append("Invalid value for verified_by")
    if len(row_remarks) == 0:
        row_remarks.append("ALL GOOD")
        set_of_galaxy_ids.add(row.galaxy_id)
    list_of_remarks.append("; ".join(row_remarks))

df["remarks"] = list_of_remarks
datasets[1][1] = df

In [26]:
# for df_constellations
df = datasets[2][1]
list_of_remarks = []
set_of_constellation_ids = set()
df_columns = list(df)

for row in df.itertuples():
    row_remarks = []
    for column in df_columns:
        if getattr(row, column) == "":
            row_remarks.append(f"Value missing for {column}")

    if row.galaxy_id and int(row.galaxy_id) not in set_of_galaxy_ids:
        row_remarks.append("Invalid value for galaxy_id")
    if row.added_by and int(row.added_by) not in set_of_user_ids:
        row_remarks.append("Invalid value for added_by")
    if row.verified_by and int(row.verified_by) not in set_of_user_ids:
        row_remarks.append("Invalid value for verified_by")
    if len(row_remarks) == 0:
        row_remarks.append("ALL GOOD")
        set_of_constellation_ids.add(row.constellation_id)
    list_of_remarks.append("; ".join(row_remarks))

df["remarks"] = list_of_remarks
datasets[2][1] = df

In [27]:
# for stars
df = datasets[3][1]
list_of_remarks = []
df_columns = list(df)

for row in df.itertuples():
    row_remarks = []
    for column in df_columns:
        if getattr(row, column) == "":
            row_remarks.append(f"Value missing for {column}")

    if row.apparent_magnitude and not 0 <= float(row.apparent_magnitude) <= 10:
        row_remarks.append("Invalid value for apparent_magnitude")
    if row.distance_mly and not 0 <= float(row.distance_mly) <= 1000:
        row_remarks.append("Invalid value for distance_mly")
    if row.constellation_id and int(row.constellation_id) not in set_of_constellation_ids:
        row_remarks.append("Invalid value for constellation_id")
    if row.added_by and int(row.added_by) not in set_of_user_ids:
        row_remarks.append("Invalid value for added_by")
    if row.verified_by and int(row.verified_by) not in set_of_user_ids:
        row_remarks.append("Invalid value for verified_by")
    if len(row_remarks) == 0:
        row_remarks.append("ALL GOOD")
    list_of_remarks.append("; ".join(row_remarks))

df["remarks"] = list_of_remarks
datasets[3][1] = df

This split approach reflects analyst practice:

- Safe fixes (like casing, whitespace, duplicates) are applied directly.
- Uncertain cases (like possible typos or implausible values) are flagged, not “magically fixed.”

This avoids introducing errors while still delivering clean, usable datasets for analysis.

Now write the flagged and cleaned dataframes into new CSV files.

In [28]:
for table, df in datasets:
    df[df["remarks"] != "ALL GOOD"].to_csv(f"data/flagged_{table}.csv", index=False)
    df[df["remarks"] == "ALL GOOD"].drop(
        columns=["remarks"]).to_csv(f"data/cleaned_{table}.csv", index=False)